#### Import dependecies

In [1]:
import random

import numpy as np
from collections import deque 
#special kind of list for the memory of our agent
#deque is like a list where you can add things from the top or end of the list
import tensorflow as tf 
from tensorflow.keras.models import Sequential # BUILD OUR NURAL NETWORK
#to aproximate optimal Q
from tensorflow.keras.layers import Dense#we will just use dense layers
from tensorflow.keras.optimizers import Adam#as optimizer we will use Adam, estoCASTIC GRADIENT DESCENT
import os #to create directories 

#### Set parameters

In [2]:
# 1. define environment

#we have 4 different state
#and 4 different actions

state_size = 4
action_size = 5
batch_size = 32 #for our gradient descent, hyperparameter you can vary for powers of two

n_episodes = 1001#number of games we want our agent to play
#number of games we play, provides as with more data for training
#in each of this episodes, we are gonna randomly remember,
#some of the things that happen in that episode
#we are gonna use that memory to train de RL agent

In [3]:
os.environ['CUDA_DEVICE_ORDER'] = "PCI_SUS_ID"#to run it in our cpu
os.environ['CUDA_VISIBLE_DEVICES'] = ""
output_dir = 'model_output/cartpole'# define a directory, to store model output

#this code if to not allow to create the directory if alreadt exists

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [4]:
import turtle

class Corona():
    
    def __init__(self):
        


        self.done = False
        self.reward = 0
        self.debt, self.spending = -2000, 6000
        

        #Background
        self.win = turtle.Screen() #create screen
        self.win.title('After Corona World') 
        self.win.bgcolor('green')
        self.win.tracer(0)
        self.win.setup(width = 600, height = 600)

        #house
        self.house = turtle.Turtle()
        self.house.shape('square')
        self.house.speed(0)
        self.house.shapesize(stretch_wid=3, stretch_len=3)   # Streach the length of square by 5 
        self.house.penup()
        self.house.color('white')       # Set the color to white
        self.house.goto(0, 0) 
        self.roof = turtle.Turtle()
        self.roof.shape('triangle')
        self.roof.tilt(-30)
        self.roof.speed(0)
        self.roof.shapesize(stretch_wid=3, stretch_len=3)   # Streach the length of square by 5 
        self.roof.penup()
        self.roof.color('red')       # Set the color to white
        self.roof.goto(0, 50-3) 

        #cinema
        self.super1 = turtle.Turtle()
        self.super1.shape('square')
        self.super1.speed(0)
        self.super1.shapesize(stretch_wid=2.5, stretch_len=4)   # Streach the length of square by 5 
        self.super1.penup()
        self.super1.color('brown')       # Set the color to white
        self.super1.goto( 160 , 0) 


        #PIZZERIA
        self.super2 = turtle.Turtle()
        self.super2.shape('square')
        self.super2.speed(0)
        self.super2.shapesize(stretch_wid=2.5, stretch_len=4)   # Streach the length of square by 5 
        self.super2.penup()
        self.super2.color('yellow')       # Set the color to white
        self.super2.goto( -160, 0 ) 

        #super3
        self.super3 = turtle.Turtle()
        self.super3.shape('square')
        self.super3.speed(0)
        self.super3.shapesize(stretch_wid=2.5, stretch_len=4)   # Streach the length of square by 5 
        self.super3.penup()
        self.super3.color('orange')       # Set the color to white
        self.super3.goto( 0, -160) 

        #cinema
        self.park = turtle.Turtle()
        self.park.shape('circle')
        self.park.speed(0)
        self.park.shapesize(stretch_wid=3.5, stretch_len=6)   # Streach the length of square by 5 
        self.park.penup()
        self.park.color('blue')       # Set the color to white
        self.park.goto( 0, 160)
        
      

        # Bilbo Beggins
        self.bilbo = turtle.Turtle()      # Create a turtle object
        self.bilbo.speed(0)
        self.bilbo.shape('circle')        # Select a circle shape
        self.bilbo.color('violet')           # Set the color to red
        self.bilbo.penup()
        self.bilbo.goto(0, 0)           # Place the shape in middle

        #names
        self.name = turtle.Turtle()
        self.name.speed(0)
        self.name.color('black')
        self.name.penup()
        self.name.hideturtle()
        
        #CINEMA
        self.name.goto(110,150)
        self.name.write("D: +20\n$: -8", align='center', font=('Courier', 14, 'normal'))
        self.name.goto(0,200)
        self.name.write("CINEMA", align='center', font=('Courier', 14, 'normal'))
        
        #PIZZERIA
        self.name.goto(160+70,-10)
        self.name.write("D: +1\n$: -20", align='center', font=('Courier', 14, 'normal'))
        self.name.goto(160, 30)
        self.name.write("PIZZERIA", align='center', font=('Courier', 14, 'normal'))        

        #COFFE SHOP
        self.name.goto(-160-70,-10)
        self.name.write("D: +6\n$: -5", align='center', font=('Courier', 14, 'normal'))
        self.name.goto(-160, 30)
        self.name.write("COFFE SHOP", align='center', font=('Courier', 14, 'normal')) 
        
        #SUPERMARKET
        self.name.goto(70,-170)
        self.name.write("D: +2\n$: -100", align='center', font=('Courier', 14, 'normal'))
        self.name.goto(0,-160+30)
        self.name.write("SUPERMARKET", align='center', font=('Courier', 14, 'normal')) 
        
        #Scorecard
        self.score = turtle.Turtle()
        self.score.speed(0)
        self.score.color('black')
        self.score.penup()
        self.score.hideturtle()
        self.score.goto(0,250)
        self.score.write("Debt: {}  Savings: {}".format(self.debt, self.spending), align='center', font=('Courier', 24, 'normal'))



    def bilbo_cinema(self):
         self.bilbo.goto(0,160)    

    def bilbo_pizzeria(self):
        self.bilbo.goto(160,0)

    def bilbo_supermarket(self):
        self.bilbo.goto(0,-160)    

    def bilbo_coffe(self):
        self.bilbo.goto(-160, 0)    
            
    def bilbo_home(self):
        self.bilbo.goto(0, 0) 

# -------------RL----------
    #ACTIONS              | REWARDS > 4000 (incentivize spend  | REWARDS < 4000((incentivize save)  
    #  0 go to cinema        ->    +2                           ->   +1  
    #  1 go to pizzeria      ->    +2                           ->   +1
    #  2 go to supermarket   ->    +1                           ->   +3
    #  3 go to coffe shop    ->    +1                           ->   +2
    #  4 go home=do nothing  ->    0                            ->   +2
    
    def reset(self):
        self.bilbo.goto(0,0)
        return [self.bilbo.xcor()*0.01, self.bilbo.ycor()*0.01, self.spending, self.debt]
    
    def step(self, action):
        
        self.reward = 0
        self.done = 0
        
        if action == 0:
            self.bilbo_cinema()
            self.reward -= .1
            
        if action == 1:
            self.bilbo_pizzeria()
            self.reward -= .1
            
        if action == 2:
            self.bilbo_supermarket()
            self.reward -= .1
            
        if action == 3:
            self.bilbo_coffe()
            self.reward -= .1  
            
        if action == 4:
            self.bilbo_home()
            self.reward -= .1  
            
            
            
        self.run_frame()
        
        state = [self.bilbo.xcor()*0.01, self.bilbo.ycor()*0.01, self.spending, self.debt]#, self.people, self.spending]
        
        #maybe put people and spendings for each supermarket?
        
        return self.reward, state, self.done
    
    
    def run_frame(self):
        
      
    
        self.win.update()
        
        
        
        if self.bilbo.pos() == (0, 160):#if Bilbo goes to the cinema
                    self.bilbo.sety(160-1)
                    self.reward += 2 *((self.spending/6000)+(-2000/(self.debt+0.00000001)))
                    self.debt += 20
                    self.spending -= 8
                    self.score.clear()
                    self.score.write("Debt: {}  Savings: {}".format(self.debt, self.spending), align='center', font=('Courier', 24, 'normal'))



        if self.bilbo.pos() == (160, 0):#if Bilbo goes to the pizzeria
                self.bilbo.setx(160-1)     
                self.reward += 3*((self.spending/6000)+(-2000/(self.debt+0.00000001))) #because he buys grosseies
                self.debt += 1
                self.spending -= 20
                self.score.clear()
                self.score.write("Debt: {}  Savings: {}".format(self.debt, self.spending), align='center', font=('Courier', 24, 'normal'))


        if self.bilbo.pos() == (0, -160):#if Bilbo goes to the Supermarket 2
                self.bilbo.sety(-160+1)
                self.reward += 2*((self.spending/6000)+(-2000/(self.debt+0.00000001)))#because he buys grosseies
                self.debt += 2
                self.spending -= 100
                self.score.clear()
                self.score.write("Debt: {}  Savings: {}".format(self.debt, self.spending), align='center', font=('Courier', 24, 'normal'))

        if self.bilbo.pos() == (-160, 0):#if Bilbo goes to the coffe shop
                self.bilbo.setx(-160+1)
                self.reward += 2*((self.spending/6000)+(-2000/(self.debt+0.00000001)))#because he buys grosseies
                self.debt += 6
                self.spending -= 5
                self.score.clear()
                self.score.write("Debt: {}  Savings: {}".format(self.debt, self.spending), align='center', font=('Courier', 24, 'normal'))


        if self.bilbo.pos() == (0, 0):#if Bilbo goes home
                self.bilbo.setx(+1)
                self.reward += 1*((self.spending/6000)+(-2000/(self.debt+0.00000001))) #because people is 0
                self.score.clear()
                self.score.write("Debt: {}  Savings: {}".format(self.debt, self.spending), align='center', font=('Courier', 24, 'normal'))
   
           

        self.x = self.spending 
        self.y = self.debt 
        if self.spending <= 500 or self.debt > 0 :#or self.people >= 4000
            self.done = True
            self.spending = 6000 #savings
            self.debt = -2000
            
        
        
            
    def end_frame(self):
        
        self.spending = 6000 #savings
        self.debt = -2000
        self.score.clear()
        self.score.write("Debt: {}  Savings: {}".format(self.debt, self.spending), align='center', font=('Courier', 24, 'normal'))

       
            
    def spend(self, x):
        
        return int(self.x)

#### Define agent

In [5]:

class DQNAgent:
    
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000) # double-ended queue; acts like list, but elements can be added/removed from either end
        self.gamma = 0.95 # decay or discount rate: enables agent to take into account future actions in addition to the immediate ones, but discounted at this rate
        self.epsilon = 1.0 # exploration rate: how much to act randomly; more initially than later due to epsilon decay
        self.epsilon_decay = 0.995 # decrease number of random explorations as the agent's performance (hopefully) improves over time
        self.epsilon_min = 0.01 # minimum amount of random exploration permitted
        self.learning_rate = 0.001 # rate at which NN adjusts models parameters via SGD to reduce cost 
        self.model = self._build_model() # private method 
    
    def _build_model(self):
        # neural net to approximate Q-value function:
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu')) # 1st hidden layer; states as input
        model.add(Dense(24, activation='relu')) # 2nd hidden layer
        model.add(Dense(self.action_size, activation='linear')) # 2 actions, so 2 output neurons: 0 and 1 (L/R)
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done)) # list of previous experiences, enabling re-training later

    def act(self, state):
        if np.random.rand() <= self.epsilon: # if acting randomly, take random action
            return random.randrange(self.action_size)
        act_values = self.model.predict(state) # if not acting randomly, predict reward value based on current state
        return np.argmax(act_values[0]) # pick the action that will give the highest reward (i.e., go left or right?)

    def replay(self, batch_size): # method that trains NN with experiences sampled from memory
        minibatch = random.sample(self.memory, batch_size) # sample a minibatch from memory
        for state, action, reward, next_state, done in minibatch: # extract data for each minibatch sample
            target = reward # if done (boolean whether game ended or not, i.e., whether final state or not), then target = reward
            if not done: # if not done, then predict future discounted reward
                target = (reward + self.gamma * # (target) = reward + (discount rate gamma) * 
                          np.amax(self.model.predict(next_state)[0])) # (maximum target Q based on future action a')
            target_f = self.model.predict(state) # approximately map current state to future discounted reward
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0) # single epoch of training with x=state, y=target_f; fit decreases loss btwn target_f and y_hat
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

In [6]:
#interacting with an OPENAI gym environment

In [7]:
agent = DQNAgent(state_size, action_size) #initialize our agent

#### Interact with environment

In [8]:

env = Corona()
done = False # episode has not ended
for e in range(n_episodes):#each episode in the range we said (2000)
    score = 0
    state = env.reset()#start eah episode at the begining f the episode
    state = state
    state = np.reshape(state, [1, state_size])#reshape this states, and transpode them so they can fit align with the neural network
    
    for time in range(30*6):#set max number of time-steps the eisode can run for,
    #max game-time is 5000 time steps
        
        #env.render() # for now doesn't work
        action = agent.act(state) #pass current state, so that it can take some initial action
        
        
        reward, next_state, done = env.step(action)
        #after our agent has taken an action
        #we can use that action to pass to the environment 
        #and get our NExT_STATE and our NEXT_REWARD from the environment
        #we also get a boolean -> if the game is done or not
        #we take that by taking an step forward
        
        score += reward  #calculate our reward
        #if you are not done, we get a penalty.
        #penalize poor actions
        
        next_state = np.reshape(next_state, [1, state_size])
        
        agent.remember(state, action, reward, next_state, done)
        #to remember the previous time-step..., all stuff we want to remember
        
        state = next_state# what state was in the previous iteration
        
        if time == 179: #if episode ended, lets print how the agent performed
            
            print('episode: {}/{}, reward: {:.4}, e: {:.2}, savings: {}, debt: {},'.format(e, n_episodes, score, agent.epsilon, env.x, env.y))
            #epsilon, good place to look if the agent is performing well 
            env.end_frame()
            
            break
        
            
        #train our data
        #give time to the agent to update his weights, so he can improves for future iterations
    if len(agent.memory) > batch_size:
        agent.replay(batch_size)
        
    if e% 50 == 0:
        agent.save(output_dir + 'weights' + '{:04d}'.format(e) + '.hdf5')#we specify format to print

episode: 0/1001, reward: 711.5, e: 1.0, savings: 2289, debt: -966,
episode: 1/1001, reward: 725.7, e: 0.99, savings: 1090, debt: -866,
episode: 2/1001, reward: 675.9, e: 0.99, savings: 802, debt: -1057,
episode: 3/1001, reward: 715.4, e: 0.99, savings: 1251, debt: -838,
episode: 4/1001, reward: 707.3, e: 0.98, savings: 1452, debt: -959,
episode: 5/1001, reward: 695.6, e: 0.98, savings: 1086, debt: -1003,
episode: 6/1001, reward: 720.6, e: 0.97, savings: 1376, debt: -893,
episode: 7/1001, reward: 743.9, e: 0.97, savings: 1457, debt: -815,
episode: 8/1001, reward: 662.6, e: 0.96, savings: 1345, debt: -1086,
episode: 9/1001, reward: 610.3, e: 0.96, savings: 5256, debt: -1899,
episode: 10/1001, reward: 766.1, e: 0.95, savings: 1924, debt: -784,
episode: 11/1001, reward: 701.4, e: 0.95, savings: 1316, debt: -890,
episode: 12/1001, reward: 818.5, e: 0.94, savings: 2144, debt: -803,
episode: 13/1001, reward: 715.0, e: 0.94, savings: 5704, debt: -1952,
episode: 14/1001, reward: 824.7, e: 0.93,

episode: 119/1001, reward: 837.0, e: 0.55, savings: 2535, debt: -872,
episode: 120/1001, reward: 744.8, e: 0.55, savings: 3386, debt: -978,
episode: 121/1001, reward: 731.7, e: 0.55, savings: 3208, debt: -996,
episode: 122/1001, reward: 852.5, e: 0.54, savings: 1154, debt: -521,
episode: 123/1001, reward: 838.6, e: 0.54, savings: 2152, debt: -779,
episode: 124/1001, reward: 750.5, e: 0.54, savings: 2091, debt: -1192,
episode: 125/1001, reward: 819.4, e: 0.53, savings: 2746, debt: -956,
episode: 126/1001, reward: 867.4, e: 0.53, savings: 2400, debt: -851,
episode: 127/1001, reward: 887.6, e: 0.53, savings: 557, debt: -281,
episode: 128/1001, reward: 757.8, e: 0.53, savings: 2302, debt: -1016,
episode: 129/1001, reward: 846.7, e: 0.52, savings: 2357, debt: -806,
episode: 130/1001, reward: 860.2, e: 0.52, savings: 2046, debt: -795,
episode: 131/1001, reward: 910.9, e: 0.52, savings: 2029, debt: -661,
episode: 132/1001, reward: 846.4, e: 0.52, savings: 1781, debt: -802,
episode: 133/1001, 

episode: 235/1001, reward: 831.8, e: 0.31, savings: 3324, debt: -1049,
episode: 236/1001, reward: 446.2, e: 0.31, savings: 1847, debt: -1535,
episode: 237/1001, reward: 763.8, e: 0.3, savings: 910, debt: -620,
episode: 238/1001, reward: 1.057e+03, e: 0.3, savings: 2361, debt: -660,
episode: 239/1001, reward: 741.8, e: 0.3, savings: 2153, debt: -1501,
episode: 240/1001, reward: 1.091e+03, e: 0.3, savings: 1079, debt: -351,
episode: 241/1001, reward: 838.3, e: 0.3, savings: 2752, debt: -1318,
episode: 242/1001, reward: 537.6, e: 0.3, savings: 1865, debt: -1372,
episode: 243/1001, reward: 847.3, e: 0.3, savings: 3327, debt: -902,
episode: 244/1001, reward: 860.8, e: 0.29, savings: 3511, debt: -1110,
episode: 245/1001, reward: 809.7, e: 0.29, savings: 3172, debt: -1141,
episode: 246/1001, reward: 822.5, e: 0.29, savings: 2237, debt: -1212,
episode: 247/1001, reward: 835.2, e: 0.29, savings: 3485, debt: -1064,
episode: 248/1001, reward: 980.7, e: 0.29, savings: 2832, debt: -915,
episode: 24

episode: 350/1001, reward: 810.4, e: 0.17, savings: 4495, debt: -919,
episode: 351/1001, reward: 798.2, e: 0.17, savings: 3514, debt: -947,
episode: 352/1001, reward: 887.6, e: 0.17, savings: 2229, debt: -1076,
episode: 353/1001, reward: 1.492e+03, e: 0.17, savings: 5790, debt: -1488,
episode: 354/1001, reward: 2.223e+03, e: 0.17, savings: 1048, debt: -77,
episode: 355/1001, reward: 1.295e+03, e: 0.17, savings: 5824, debt: -1560,
episode: 356/1001, reward: 788.0, e: 0.17, savings: 3410, debt: -984,
episode: 357/1001, reward: 410.5, e: 0.17, savings: 4967, debt: -1803,
episode: 358/1001, reward: 1.351e+03, e: 0.17, savings: 4269, debt: -218,
episode: 359/1001, reward: 1.338e+03, e: 0.17, savings: 5719, debt: -1552,
episode: 360/1001, reward: 847.4, e: 0.16, savings: 2699, debt: -1598,
episode: 361/1001, reward: 4.09e+03, e: 0.16, savings: 5643, debt: -1610,
episode: 362/1001, reward: 2.033e+03, e: 0.16, savings: 4541, debt: -105,
episode: 363/1001, reward: 817.4, e: 0.16, savings: 4576,

episode: 464/1001, reward: 368.3, e: 0.098, savings: 5598, debt: -1899,
episode: 465/1001, reward: 907.4, e: 0.097, savings: 2375, debt: -1721,
episode: 466/1001, reward: 979.3, e: 0.097, savings: 4249, debt: -201,
episode: 467/1001, reward: 917.4, e: 0.096, savings: 2402, debt: -1792,
episode: 468/1001, reward: 860.7, e: 0.096, savings: 3548, debt: -1345,
episode: 469/1001, reward: 588.4, e: 0.095, savings: 1281, debt: -1072,
episode: 470/1001, reward: 627.7, e: 0.095, savings: 1770, debt: -1120,
episode: 471/1001, reward: 669.8, e: 0.094, savings: 2318, debt: -1031,
episode: 472/1001, reward: 703.0, e: 0.094, savings: 2476, debt: -1018,
episode: 473/1001, reward: 564.7, e: 0.093, savings: 3378, debt: -1792,
episode: 474/1001, reward: 694.2, e: 0.093, savings: 2521, debt: -1026,
episode: 475/1001, reward: 369.8, e: 0.092, savings: 4581, debt: -1869,
episode: 476/1001, reward: 604.2, e: 0.092, savings: 3297, debt: -1740,
episode: 477/1001, reward: 701.1, e: 0.092, savings: 2682, debt: 

episode: 576/1001, reward: 1.361e+03, e: 0.056, savings: 5348, debt: -1610,
episode: 577/1001, reward: 1.432e+03, e: 0.055, savings: 5460, debt: -1394,
episode: 578/1001, reward: 365.0, e: 0.055, savings: 5623, debt: -1906,
episode: 579/1001, reward: 889.0, e: 0.055, savings: 2414, debt: -1390,
episode: 580/1001, reward: 789.5, e: 0.055, savings: 2172, debt: -1469,
episode: 581/1001, reward: 927.3, e: 0.054, savings: 4949, debt: -703,
episode: 582/1001, reward: 368.6, e: 0.054, savings: 5521, debt: -1912,
episode: 583/1001, reward: 876.1, e: 0.054, savings: 2778, debt: -1594,
episode: 584/1001, reward: 879.3, e: 0.054, savings: 5435, debt: -1966,
episode: 585/1001, reward: 1.144e+03, e: 0.053, savings: 2642, debt: -741,
episode: 586/1001, reward: 1.153e+03, e: 0.053, savings: 2843, debt: -787,
episode: 587/1001, reward: 609.8, e: 0.053, savings: 1586, debt: -1061,
episode: 588/1001, reward: 888.1, e: 0.052, savings: 4181, debt: -1149,
episode: 589/1001, reward: 791.3, e: 0.052, savings

episode: 690/1001, reward: 869.7, e: 0.031, savings: 3671, debt: -1339,
episode: 691/1001, reward: 858.2, e: 0.031, savings: 3199, debt: -1476,
episode: 692/1001, reward: 854.6, e: 0.031, savings: 3335, debt: -1496,
episode: 693/1001, reward: 901.7, e: 0.031, savings: 1615, debt: -1805,
episode: 694/1001, reward: 860.0, e: 0.031, savings: 3705, debt: -1368,
episode: 695/1001, reward: 863.2, e: 0.031, savings: 3375, debt: -1495,
episode: 696/1001, reward: 866.6, e: 0.031, savings: 3443, debt: -1357,
episode: 697/1001, reward: 887.7, e: 0.03, savings: 3947, debt: -1297,
episode: 698/1001, reward: 2.027e+03, e: 0.03, savings: 5592, debt: -1136,
episode: 699/1001, reward: 870.9, e: 0.03, savings: 3474, debt: -1432,
episode: 700/1001, reward: 866.3, e: 0.03, savings: 3366, debt: -1433,
episode: 701/1001, reward: 881.7, e: 0.03, savings: 1087, debt: -1780,
episode: 702/1001, reward: 900.3, e: 0.03, savings: 1859, debt: -1772,
episode: 703/1001, reward: 878.8, e: 0.029, savings: 3827, debt: -

episode: 803/1001, reward: 971.1, e: 0.018, savings: 2832, debt: -1136,
episode: 804/1001, reward: 949.5, e: 0.018, savings: 2864, debt: -1193,
episode: 805/1001, reward: 966.4, e: 0.018, savings: 2832, debt: -1136,
episode: 806/1001, reward: 954.5, e: 0.018, savings: 2796, debt: -1193,
episode: 807/1001, reward: 958.7, e: 0.018, savings: 2752, debt: -1135,
episode: 808/1001, reward: 959.6, e: 0.017, savings: 2699, debt: -1110,
episode: 809/1001, reward: 855.1, e: 0.017, savings: 3430, debt: -1449,
episode: 810/1001, reward: 861.8, e: 0.017, savings: 3450, debt: -1470,
episode: 811/1001, reward: 949.2, e: 0.017, savings: 2849, debt: -1198,
episode: 812/1001, reward: 945.4, e: 0.017, savings: 2743, debt: -1168,
episode: 813/1001, reward: 935.7, e: 0.017, savings: 2668, debt: -1173,
episode: 814/1001, reward: 931.7, e: 0.017, savings: 2862, debt: -1276,
episode: 815/1001, reward: 948.5, e: 0.017, savings: 2834, debt: -1203,
episode: 816/1001, reward: 854.3, e: 0.017, savings: 3079, debt:

episode: 917/1001, reward: 875.5, e: 0.01, savings: 2899, debt: -1506,
episode: 918/1001, reward: 819.8, e: 0.01, savings: 3315, debt: -1349,
episode: 919/1001, reward: 915.7, e: 0.01, savings: 2699, debt: -1398,
episode: 920/1001, reward: 807.6, e: 0.01, savings: 3085, debt: -1398,
episode: 921/1001, reward: 909.9, e: 0.01, savings: 2632, debt: -1518,
episode: 922/1001, reward: 914.9, e: 0.01, savings: 2532, debt: -1611,
episode: 923/1001, reward: 857.7, e: 0.01, savings: 3295, debt: -1494,
episode: 924/1001, reward: 912.7, e: 0.01, savings: 2572, debt: -1613,
episode: 925/1001, reward: 907.8, e: 0.01, savings: 2500, debt: -1439,
episode: 926/1001, reward: 910.7, e: 0.01, savings: 2600, debt: -1536,
episode: 927/1001, reward: 914.7, e: 0.01, savings: 2500, debt: -1727,
episode: 928/1001, reward: 920.3, e: 0.01, savings: 2508, debt: -1649,
episode: 929/1001, reward: 922.4, e: 0.01, savings: 2335, debt: -1814,
episode: 930/1001, reward: 922.1, e: 0.01, savings: 2320, debt: -1819,
episod

In [ ]:
def train_dqn(n_episodes, state_size, action_size):

    loss = []
    
    agent = DQNAgent(state_size, action_size) #initialize our agent
    
    env = Corona()
    done = False # episode has not ended
    for e in range(n_episodes):#each episode in the range we said (2000)

        state = env.reset()#start eah episode at the begining f the episode
        state = state
        state = np.reshape(state, [1, state_size])#reshape this states, and transpode them so they can fit align with the neural network

        for time in range(5000):#set max number of time-steps the eisode can run for,
        #max game-time is 5000 time steps

            #env.render() # for now doesn't work
            action = agent.act(state) #pass current state, so that it can take some initial action


            reward, next_state, done = env.step(action)
            #after our agent has taken an action
            #we can use that action to pass to the environment 
            #and get our NExT_STATE and our NEXT_REWARD from the environment
            #we also get a boolean -> if the game is done or not
            #we take that by taking an step forward

            reward = reward if not done else -0.1#calculate our reward
            #if you are not done, we get a penalty.
            #penalize poor actions

            next_state = np.reshape(next_state, [1, state_size])

            agent.remember(state, action, reward, next_state, done)
            #to remember the previous time-step..., all stuff we want to remember

            state = next_state# what state was in the previous iteration

            if done: #if episode ended, lets print how the agent performed
                
                
                print('episode: {}/{}, score: {}, e: {:.2}'.format(e, n_episodes, time, agent.epsilon))
                #epsilon, good place to look if the agent is performing well
                break

            #train our data
            #give time to the agent to update his weights, so he can improves for future iterations
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)

        if e% 50 == 0:
            agent.save(output_dir + 'weights' + '{:04d}'.format(e) + '.hdf5')#we specify format to print
            ------
    
        loss.append(reward)
    return loss


if __name__ == '__main__':

    ep = 1001
    loss = train_dqn(ep,  state_size, action_size)
    plt.plot([i for i in range(ep)], loss)
    plt.xlabel('episodes')
    plt.ylabel('reward')
    plt.show()

In [ ]:
 for time in range(30*6):#set max number of time-steps the eisode can run for,

            print(time)
            
    #if time = 30: #if episode ended, lets print how the agent performed
            
       # print('ok')

In [8]:
spending =(6000,5000, 4000, 3000, 2000, 1000, 500, 200, 100)
for i in spending:
    print(2*(i/6000))

2.0
1.6666666666666667
1.3333333333333333
1.0
0.6666666666666666
0.3333333333333333
0.16666666666666666
0.06666666666666667
0.03333333333333333


In [14]:
spending =(-2000,-1900, -1800, -1000, -500)
for i in spending:
    print((-2000/i))

1.0
1.0526315789473684
1.1111111111111112
2.0
4.0
